## Observations and Insights 

In [1]:
# Dependencies and Setup
import pandas as pd
import scipy.stats as st
import numpy as np
from sqlalchemy import create_engine
import psycopg2



In [2]:
# Study data files
olympic_path = "data/athlete_events.csv"
noc_path= "data/noc_country.csv"
economic_path = "data/w_gdp.csv"
population_path = "data/world_pop.csv"
host_path = "data/host_country.csv"

# Read the data and olympic results
olympic_data = pd.read_csv(olympic_path)
noc_data = pd.read_csv(noc_path)
economic_data = pd.read_csv(economic_path)
population_data= pd.read_csv(population_path)
host_data= pd.read_csv(host_path)

# Combine the data into a single dataset

# Display the data table for preview

In [3]:
olympic_data.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [4]:
olympic_data.shape

(271116, 15)

In [5]:
olympic_data.dtypes

ID          int64
Name       object
Sex        object
Age       float64
Height    float64
Weight    float64
Team       object
NOC        object
Games      object
Year        int64
Season     object
City       object
Sport      object
Event      object
Medal      object
dtype: object

In [6]:
#Begin Transforming Olympic Medal data by dropping all rows associated with Winter Olympics
olympic_data.drop(olympic_data[olympic_data["Season"]=="Winter"].index, inplace=True)

In [7]:
olympic_data.shape

(222552, 15)

In [8]:
#Since we are confining our analysis to 1960 onwards, we will drop all rows associated with games pre-1960
olympic_data.drop(olympic_data[olympic_data["Year"]<1960].index, inplace=True)

In [9]:
olympic_data.shape

(166267, 15)

In [10]:
olympic_data.head(100)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
31,12,Jyri Tapani Aalto,M,31.0,172.0,70.0,Finland,FIN,2000 Summer,2000,Summer,Sydney,Badminton,Badminton Men's Singles,NaN
32,13,Minna Maarit Aalto,F,30.0,159.0,55.5,Finland,FIN,1996 Summer,1996,Summer,Atlanta,Sailing,Sailing Women's Windsurfer,NaN
33,13,Minna Maarit Aalto,F,34.0,159.0,55.5,Finland,FIN,2000 Summer,2000,Summer,Sydney,Sailing,Sailing Women's Windsurfer,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,101,Adeyemi Abayomi,M,25.0,170.0,51.0,Nigeria,NGR,1972 Summer,1972,Summer,Munich,Boxing,Boxing Men's Lightweight,NaN
215,104,Gana Abba Kimet,M,26.0,NaN,NaN,Chad,CHA,1972 Summer,1972,Summer,Munich,Athletics,Athletics Men's 100 metres,NaN
216,105,Ilyas Abbadi,M,19.0,175.0,75.0,Algeria,ALG,2012 Summer,2012,Summer,London,Boxing,Boxing Men's Welterweight,NaN
217,105,Ilyas Abbadi,M,23.0,175.0,75.0,Algeria,ALG,2016 Summer,2016,Summer,Rio de Janeiro,Boxing,Boxing Men's Middleweight,NaN


In [11]:
#To simplify our data we will drop columns not relevant to our Project, namely: Name, Age, Height, cont.
#Season (all winter games previously dropped), sport and event
olympic_data=olympic_data.drop(["Name", "Age", "Height", "Weight", "Sport", "Event", "Sex", "Season", "Games", "Team"], axis=1)

In [12]:
#Instead of pivoting out Medal column, will create a series of conditional columns, based on whether the medal was gold, silver or bronze
olympic_data['Gold'] = np.where(olympic_data['Medal']=='Gold', 1, 0)
olympic_data['Silver'] = np.where(olympic_data['Medal']=='Silver', 1, 0)
olympic_data['Bronze'] = np.where(olympic_data['Medal']=='Bronze', 1, 0)

In [13]:
olympic_data.head(100)

,ID,NOC,Year,City,Medal,Gold,Silver,Bronze
0,1,CHN,1992,Barcelona,NaN,0,0,0
1,2,CHN,2012,London,NaN,0,0,0
31,12,FIN,2000,Sydney,NaN,0,0,0
32,13,FIN,1996,Atlanta,NaN,0,0,0
33,13,FIN,2000,Sydney,NaN,0,0,0
...,...,...,...,...,...,...,...,...
211,101,NGR,1972,Munich,NaN,0,0,0
215,104,CHA,1972,Munich,NaN,0,0,0
216,105,ALG,2012,London,NaN,0,0,0
217,105,ALG,2016,Rio de Janeiro,NaN,0,0,0


In [14]:
olympic_data["Medal"] = olympic_data["Medal"].fillna(0)

In [15]:
olympic_data.loc[(olympic_data.Medal != 0),'Medal']=1

In [16]:
olympic_data

,ID,NOC,Year,City,Medal,Gold,Silver,Bronze
0,1,CHN,1992,Barcelona,0,0,0,0
1,2,CHN,2012,London,0,0,0,0
31,12,FIN,2000,Sydney,0,0,0,0
32,13,FIN,1996,Atlanta,0,0,0,0
33,13,FIN,2000,Sydney,0,0,0,0
...,...,...,...,...,...,...,...,...
271106,135565,ARG,2004,Athina,0,0,0,0
271107,135566,USA,1972,Munich,0,0,0,0
271108,135567,RUS,2000,Sydney,0,0,0,0
271109,135567,RUS,2004,Athina,0,0,0,0


In [17]:
# Before we join with our masterlist which maps country names, NOC and World Bank identifiers, we will check to see if there are any missing values

In [18]:
#Check to see if there are any missing values for any of our columns.
print(olympic_data.isnull().sum())

ID        0
NOC       0
Year      0
City      0
Medal     0
Gold      0
Silver    0
Bronze    0
dtype: int64


In [19]:
olympic_data=pd.merge(olympic_data, noc_data, how="left", on=["NOC", "NOC"])

In [20]:
olympic_data.head()

,ID,NOC,Year,City,Medal,Gold,Silver,Bronze,country,notes,wb_country_code
0,1,CHN,1992,Barcelona,0,0,0,0,China,NaN,CHN
1,2,CHN,2012,London,0,0,0,0,China,NaN,CHN
2,12,FIN,2000,Sydney,0,0,0,0,Finland,NaN,FIN
3,13,FIN,1996,Atlanta,0,0,0,0,Finland,NaN,FIN
4,13,FIN,2000,Sydney,0,0,0,0,Finland,NaN,FIN


In [21]:
olympic_data=olympic_data.drop(["notes"], axis=1)

In [22]:
#Check to see if there are any missing values for any of our columns.
print(olympic_data.isnull().sum())

ID                    0
NOC                   0
Year                  0
City                  0
Medal                 0
Gold                  0
Silver                0
Bronze                0
country             444
wb_country_code    1621
dtype: int64


In [23]:
no_country=olympic_data[olympic_data.country.isnull()]
no_country

,ID,NOC,Year,City,Medal,Gold,Silver,Bronze,country,wb_country_code
119,116,UAR,1960,Roma,0,0,0,0,NaN,NaN
169,163,UAR,1960,Roma,0,0,0,0,NaN,NaN
170,163,UAR,1960,Roma,0,0,0,0,NaN,NaN
171,163,UAR,1960,Roma,0,0,0,0,NaN,NaN
172,163,UAR,1960,Roma,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
164733,134454,SGP,2004,Athina,0,0,0,0,NaN,NaN
164734,134454,SGP,2008,Beijing,0,0,0,0,NaN,NaN
164934,134587,SGP,2004,Athina,0,0,0,0,NaN,NaN
164935,134587,SGP,2004,Athina,0,0,0,0,NaN,NaN


In [24]:
#So countries with National Organising Committee codes SGP, TUV and ROT seem to be missing from our NOC to country mapping list
#UAR is a defunct temporary political union between Syria and Egypt in the late 60s and early 70s- we will exclude it from our list of countries
#After further investigation SGP is Singapore, TUV is Tuvalu and ROT is Refugee Olympic Athletes. We will go in and manually enter these values
olympic_data['country'] = np.where(olympic_data['NOC']=='SGP', 'Singapore', olympic_data['country'])
olympic_data['country'] = np.where(olympic_data['NOC']=='TUV', 'Tuvalu', olympic_data['country'])
olympic_data['country'] = np.where(olympic_data['NOC']=='ROT', 'Refugee Athletes', olympic_data['country'])

In [25]:
print(olympic_data.isnull().sum())

ID                    0
NOC                   0
Year                  0
City                  0
Medal                 0
Gold                  0
Silver                0
Bronze                0
country             143
wb_country_code    1621
dtype: int64


In [26]:
#Quickly look to see which countries are missing World Bank country codes

In [27]:
no_country_code=olympic_data[olympic_data.wb_country_code.isnull()]
no_country_code

,ID,NOC,Year,City,Medal,Gold,Silver,Bronze,country,wb_country_code
119,116,UAR,1960,Roma,0,0,0,0,NaN,NaN
165,160,ERI,2008,Beijing,0,0,0,0,Eritrea,NaN
169,163,UAR,1960,Roma,0,0,0,0,NaN,NaN
170,163,UAR,1960,Roma,0,0,0,0,NaN,NaN
171,163,UAR,1960,Roma,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
164934,134587,SGP,2004,Athina,0,0,0,0,Singapore,NaN
164935,134587,SGP,2004,Athina,0,0,0,0,Singapore,NaN
165306,134826,SGP,2016,Rio de Janeiro,0,0,0,0,Singapore,NaN
165563,135019,PLE,2016,Rio de Janeiro,0,0,0,0,Palestine,NaN


In [28]:
#Hmmmmm.... We can add in a World Bank country code for Singapore, like we did for Singapore's NOC. 
#Unfortunately, for the other countries it appears the World Bank does not have GDP or population data,
#In the case of Taiwan, this seems to be a common frustration: https://datahelpdesk.worldbank.org/knowledgebase/articles/114933-where-are-your-data-on-taiwan
#In other countries, we may attribute the lacking country codes due to non-recognition (Cook Islands, Palestine), or lack of data due to conflict (South Sudan, Eritrea)
#Where countries do not have World Bank data, we will exclude them from further analysis

In [29]:
#Fix country code for Singapore
olympic_data['wb_country_code'] = np.where(olympic_data['NOC']=='SGP', 'SGP', olympic_data['wb_country_code'])
#drop observations for countries without World Bank country codes or data
olympic_data["wb_country_code"] = olympic_data["wb_country_code"].fillna(0)
olympic_data.drop(olympic_data[olympic_data["wb_country_code"]==0].index, inplace=True)

In [30]:
olympic_data

,ID,NOC,Year,City,Medal,Gold,Silver,Bronze,country,wb_country_code
0,1,CHN,1992,Barcelona,0,0,0,0,China,CHN
1,2,CHN,2012,London,0,0,0,0,China,CHN
2,12,FIN,2000,Sydney,0,0,0,0,Finland,FIN
3,13,FIN,1996,Atlanta,0,0,0,0,Finland,FIN
4,13,FIN,2000,Sydney,0,0,0,0,Finland,FIN
...,...,...,...,...,...,...,...,...,...,...
166262,135565,ARG,2004,Athina,0,0,0,0,Argentina,ARG
166263,135566,USA,1972,Munich,0,0,0,0,USA,USA
166264,135567,RUS,2000,Sydney,0,0,0,0,Russia,RUS
166265,135567,RUS,2004,Athina,0,0,0,0,Russia,RUS


In [31]:
no_country_code=olympic_data[olympic_data.wb_country_code.isnull()]
no_country_code


,ID,NOC,Year,City,Medal,Gold,Silver,Bronze,country,wb_country_code


In [32]:
olympic_data.head()

,ID,NOC,Year,City,Medal,Gold,Silver,Bronze,country,wb_country_code
0,1,CHN,1992,Barcelona,0,0,0,0,China,CHN
1,2,CHN,2012,London,0,0,0,0,China,CHN
2,12,FIN,2000,Sydney,0,0,0,0,Finland,FIN
3,13,FIN,1996,Atlanta,0,0,0,0,Finland,FIN
4,13,FIN,2000,Sydney,0,0,0,0,Finland,FIN


In [33]:
olympic_data=olympic_data.drop(["NOC"], axis=1)

In [34]:
olympic_data.head()

,ID,Year,City,Medal,Gold,Silver,Bronze,country,wb_country_code
0,1,1992,Barcelona,0,0,0,0,China,CHN
1,2,2012,London,0,0,0,0,China,CHN
2,12,2000,Sydney,0,0,0,0,Finland,FIN
3,13,1996,Atlanta,0,0,0,0,Finland,FIN
4,13,2000,Sydney,0,0,0,0,Finland,FIN


In [35]:
#-----------------------------------------------------------------------------------------------------------------------

In [36]:
#Now we will aggregate by Year, Host city and by World Bank country code

In [37]:
NOC_games_summary=pd.DataFrame(olympic_data.groupby(["Year", "City", "wb_country_code"]).agg({"ID": "count", "Medal":"sum", "Gold":"sum","Silver":"sum", "Bronze":"sum"}).reset_index())

In [38]:
NOC_games_summary.head()

,Year,City,wb_country_code,ID,Medal,Gold,Silver,Bronze
0,1960,Roma,AFG,16,0,0,0,0
1,1960,Roma,ARG,116,4,0,3,1
2,1960,Roma,AUS,280,46,11,24,11
3,1960,Roma,AUT,198,3,1,2,0
4,1960,Roma,BEL,150,4,0,2,2


In [39]:
NOC_games_summary.shape

(2209, 8)

In [40]:
NOC_games_summary = NOC_games_summary.rename(columns={'ID': 'No. of Athletes'})

In [41]:
NOC_games_summary.head()

,Year,City,wb_country_code,No. of Athletes,Medal,Gold,Silver,Bronze
0,1960,Roma,AFG,16,0,0,0,0
1,1960,Roma,ARG,116,4,0,3,1
2,1960,Roma,AUS,280,46,11,24,11
3,1960,Roma,AUT,198,3,1,2,0
4,1960,Roma,BEL,150,4,0,2,2


In [42]:
country_list=NOC_games_summary["wb_country_code"].unique().tolist()

In [43]:
country_list

['AFG',
 'ARG',
 'AUS',
 'AUT',
 'BEL',
 'BGR',
 'BHS',
 'BMU',
 'BRA',
 'CAN',
 'CHE',
 'CHL',
 'CHN',
 'COL',
 'CUB',
 'CUW',
 'CZE',
 'DEU',
 'DNK',
 'ESP',
 'ETH',
 'FIN',
 'FJI',
 'FRA',
 'GBR',
 'GHA',
 'GRC',
 'GUY',
 'HTI',
 'HUN',
 'IDN',
 'IND',
 'IRL',
 'IRN',
 'IRQ',
 'ISL',
 'ISR',
 'ITA',
 'JPN',
 'KEN',
 'KOR',
 'LBN',
 'LBR',
 'LIE',
 'LKA',
 'LUX',
 'MAR',
 'MCO',
 'MEX',
 'MLT',
 'MMR',
 'MYS',
 'NGA',
 'NLD',
 'NOR',
 'NZL',
 'PAK',
 'PAN',
 'PER',
 'PHL',
 'POL',
 'PRI',
 'PRT',
 'ROU',
 'RUS',
 'SDN',
 'SGP',
 'SMR',
 'SRB',
 'SWE',
 'THA',
 'TUN',
 'TUR',
 'UGA',
 'URY',
 'USA',
 'VEN',
 'VNM',
 'ZAF',
 'ZWE',
 'BOL',
 'CIV',
 'CMR',
 'COG',
 'CRI',
 'DOM',
 'DZA',
 'EGY',
 'JAM',
 'KHM',
 'MDG',
 'MLI',
 'MNG',
 'NER',
 'NPL',
 'SEN',
 'TCD',
 'TTO',
 'TZA',
 'ZMB',
 'BLZ',
 'BRB',
 'CAF',
 'COD',
 'ECU',
 'GIN',
 'GTM',
 'HND',
 'KWT',
 'LBY',
 'NIC',
 'PRY',
 'SLE',
 'SLV',
 'SUR',
 'SYR',
 'VIR',
 'ALB',
 'BEN',
 'BFA',
 'GAB',
 'LSO',
 'MWI',
 'PRK',
 'SAU',


In [44]:
len(country_list)

201

In [45]:
#Much better! Now need to bring in the GDP Data


In [46]:
economic_data.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,1.019053e+10,1.248694e+10,1.593680e+10,1.793024e+10,2.053654e+10,2.004633e+10,2.005019e+10,1.921556e+10,1.946902e+10,NaN
1,Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.288135e+10,1.204421e+10,1.192695e+10,1.289087e+10,1.231978e+10,1.277628e+10,1.322824e+10,1.133526e+10,1.186387e+10,NaN
2,Algeria,DZA,GDP (current US$),NY.GDP.MKTP.CD,2.723649e+09,2.434777e+09,2.001469e+09,2.703015e+09,2.909352e+09,3.136259e+09,...,1.710010e+11,1.372110e+11,1.612070e+11,2.000190e+11,2.090590e+11,2.097550e+11,2.138100e+11,1.658740e+11,1.590490e+11,NaN
3,American Samoa,ASM,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,5.630000e+08,6.780000e+08,5.760000e+08,5.740000e+08,6.440000e+08,6.410000e+08,6.430000e+08,6.590000e+08,6.580000e+08,NaN
4,Andorra,AND,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,4.007353e+09,3.660531e+09,3.355695e+09,3.442063e+09,3.164615e+09,3.281585e+09,3.350736e+09,2.811489e+09,2.858518e+09,NaN


In [47]:
#Years and GDP are in columns for each country, when we want them in rows for our economic data. Will have to pivot data into 
#more suitable shape before merging
economic_data=economic_data.melt(id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"], 
        var_name="Year", 
        value_name="GDP")

In [48]:
economic_data.dtypes

Country Name       object
Country Code       object
Indicator Name     object
Indicator Code     object
Year               object
GDP               float64
dtype: object

In [49]:
economic_data["Year"] = economic_data["Year"].astype(str).astype(int)

In [50]:
economic_data.dtypes

Country Name       object
Country Code       object
Indicator Name     object
Indicator Code     object
Year                int32
GDP               float64
dtype: object

In [51]:
#Check for duplicates before merging economic data and olympic data

In [52]:
eco_dupes =economic_data[economic_data.duplicated()]
eco_dupes

,Country Name,Country Code,Indicator Name,Indicator Code,Year,GDP


In [53]:
olympic_dupes =NOC_games_summary[NOC_games_summary.duplicated()]
olympic_dupes

,Year,City,wb_country_code,No. of Athletes,Medal,Gold,Silver,Bronze


In [54]:
#No duplicates, so let's merge

In [55]:
country_data_complete=pd.merge(NOC_games_summary, economic_data, how="left", left_on=["wb_country_code", "Year"], right_on=["Country Code", "Year"])

In [56]:
country_data_complete=country_data_complete.drop(["Country Name", "Indicator Code", "Indicator Name"], axis=1)

In [57]:
country_data_complete.head()

,Year,City,wb_country_code,No. of Athletes,Medal,Gold,Silver,Bronze,Country Code,GDP
0,1960,Roma,AFG,16,0,0,0,0,AFG,5.377778e+08
1,1960,Roma,ARG,116,4,0,3,1,ARG,NaN
2,1960,Roma,AUS,280,46,11,24,11,AUS,1.859335e+10
3,1960,Roma,AUT,198,3,1,2,0,AUT,6.592694e+09
4,1960,Roma,BEL,150,4,0,2,2,BEL,1.165872e+10


In [58]:
#Now to merge in population data

In [59]:
population_data.head()

,Country,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,101220.0,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,...,26616792.0,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032
2,Angola,AGO,"Population, total",SP.POP.TOTL,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,6203299.0,...,20997687.0,21759420.0,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,82683.0,83861.0,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281


In [60]:
#Like GPD, Years and population are in columns for each country, when we want them in rows for our economic data. Will have to pivot data into 
#more suitable shape before merging
population_data=population_data.melt(id_vars=["Country", "Country Code", "Indicator Name", "Indicator Code"], 
        var_name="Year", 
        value_name="Population")

In [61]:
population_data.head()

,Country,Country Code,Indicator Name,Indicator Code,Year,Population
0,Aruba,ABW,"Population, total",SP.POP.TOTL,1960,54211.0
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,1960,8996351.0
2,Angola,AGO,"Population, total",SP.POP.TOTL,1960,5643182.0
3,Albania,ALB,"Population, total",SP.POP.TOTL,1960,1608800.0
4,Andorra,AND,"Population, total",SP.POP.TOTL,1960,13411.0


In [62]:
population_data.dtypes

Country            object
Country Code       object
Indicator Name     object
Indicator Code     object
Year               object
Population        float64
dtype: object

In [63]:
population_data["Year"] = population_data["Year"].astype(str).astype(int)

In [64]:
country_data_complete=pd.merge(country_data_complete, population_data, how="left", left_on=["wb_country_code", "Year"], right_on=["Country Code", "Year"])

In [65]:
country_data_complete.head()

,Year,City,wb_country_code,No. of Athletes,Medal,Gold,Silver,Bronze,Country Code_x,GDP,Country,Country Code_y,Indicator Name,Indicator Code,Population
0,1960,Roma,AFG,16,0,0,0,0,AFG,5.377778e+08,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.0
1,1960,Roma,ARG,116,4,0,3,1,ARG,NaN,Argentina,ARG,"Population, total",SP.POP.TOTL,20619075.0
2,1960,Roma,AUS,280,46,11,24,11,AUS,1.859335e+10,Australia,AUS,"Population, total",SP.POP.TOTL,10276477.0
3,1960,Roma,AUT,198,3,1,2,0,AUT,6.592694e+09,Austria,AUT,"Population, total",SP.POP.TOTL,7047539.0
4,1960,Roma,BEL,150,4,0,2,2,BEL,1.165872e+10,Belgium,BEL,"Population, total",SP.POP.TOTL,9153489.0


In [66]:
country_data_complete=country_data_complete.drop(["Indicator Name", "Indicator Code", "Country Code_x", "Country Code_y"], axis=1)

In [67]:
#Host Country

In [68]:
country_data_complete=pd.merge(country_data_complete, host_data, how="left", left_on=["Country", "Year"], right_on=["Host_Country", "Year"])

In [69]:
country_data_complete.head()

,Year,City_x,wb_country_code,No. of Athletes,Medal,Gold,Silver,Bronze,GDP,Country,Population,City_y,Host_Country,Host
0,1960,Roma,AFG,16,0,0,0,0,5.377778e+08,Afghanistan,8996351.0,NaN,NaN,NaN
1,1960,Roma,ARG,116,4,0,3,1,NaN,Argentina,20619075.0,NaN,NaN,NaN
2,1960,Roma,AUS,280,46,11,24,11,1.859335e+10,Australia,10276477.0,NaN,NaN,NaN
3,1960,Roma,AUT,198,3,1,2,0,6.592694e+09,Austria,7047539.0,NaN,NaN,NaN
4,1960,Roma,BEL,150,4,0,2,2,1.165872e+10,Belgium,9153489.0,NaN,NaN,NaN


In [70]:
country_data_complete["Host"] = country_data_complete["Host"].fillna(0)

In [71]:
country_data_complete.dtypes

Year                 int64
City_x              object
wb_country_code     object
No. of Athletes      int64
Medal                int64
Gold                 int32
Silver               int32
Bronze               int32
GDP                float64
Country             object
Population         float64
City_y              object
Host_Country        object
Host               float64
dtype: object

In [72]:
print(country_data_complete.isnull().sum())

Year                  0
City_x                0
wb_country_code       0
No. of Athletes       0
Medal                 0
Gold                  0
Silver                0
Bronze                0
GDP                 236
Country               0
Population            9
City_y             2199
Host_Country       2199
Host                  0
dtype: int64


In [73]:
no_GDP=country_data_complete[country_data_complete.GDP.isnull()]
no_GDP

,Year,City_x,wb_country_code,No. of Athletes,Medal,Gold,Silver,Bronze,GDP,Country,Population,City_y,Host_Country,Host
1,1960,Roma,ARG,116,4,0,3,1,NaN,Argentina,20619075.0,NaN,NaN,0.0
5,1960,Roma,BGR,183,7,1,3,3,NaN,Bulgaria,7867374.0,NaN,NaN,0.0
14,1960,Roma,CUB,21,0,0,0,0,NaN,Cuba,7141135.0,NaN,NaN,0.0
15,1960,Roma,CUW,5,0,0,0,0,NaN,Curacao,124826.0,NaN,NaN,0.0
16,1960,Roma,CZE,202,22,4,7,11,NaN,Czech Republic,9602006.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,2016,Rio de Janeiro,PRK,38,7,2,3,2,NaN,"Korea, Dem. People Rep.",25368620.0,NaN,NaN,0.0
2179,2016,Rio de Janeiro,SYR,7,0,0,0,0,NaN,Syrian Arab Republic,18430453.0,NaN,NaN,0.0
2198,2016,Rio de Janeiro,VEN,105,3,0,1,2,NaN,"Venezuela, RB",31568179.0,NaN,NaN,0.0
2199,2016,Rio de Janeiro,VGB,4,0,0,0,0,NaN,British Virgin Islands,30661.0,NaN,NaN,0.0


In [74]:
no_GDP_list=no_GDP["Country"].unique().tolist()
no_GDP_list

['Argentina',
 'Bulgaria',
 'Cuba',
 'Curacao',
 'Czech Republic',
 'Germany',
 'Ethiopia',
 'Haiti',
 'Hungary',
 'Indonesia',
 'Lebanon',
 'Liechtenstein',
 'Monaco',
 'Malta',
 'Myanmar',
 'Poland',
 'Romania',
 'Russia',
 'San Marino',
 'Serbia',
 'Tunisia',
 'Vietnam',
 'Egypt',
 'Mali',
 'Mongolia',
 'Tanzania',
 'Barbados',
 'Guinea',
 'Libya',
 'Nicaragua',
 'Albania',
 'Switzerland',
 'Korea, Dem. People Rep.',
 'Somalia',
 'Antigua and Barbuda',
 'Cayman Islands',
 'Angola',
 'Lao PDR',
 'Djibouti',
 'British Virgin Islands',
 'Yemen, Rep.',
 'Aruba',
 'Afghanistan',
 'American Samoa',
 'Guam',
 'Bosnia and Herzegovina',
 'Estonia',
 'Croatia',
 'Iran',
 'Iraq',
 'Lithuania',
 'Latvia',
 'Slovenia',
 'Nauru',
 'Sao Tome and Principe',
 'Virgin Islands (U.S.)',
 'Syrian Arab Republic',
 'Bermuda',
 'Puerto Rico',
 'Venezuela, RB']

In [75]:
#Inspecting the original CSV files for GDP, it seems these countries do have missing values for GDP, including Germany (which is surprsing)

In [76]:
#A quick check to make sure we have correctly applied the host variable
is_host = country_data_complete[country_data_complete['Host']==1]
is_host.head(20)

,Year,City_x,wb_country_code,No. of Athletes,Medal,Gold,Silver,Bronze,GDP,Country,Population,City_y,Host_Country,Host
37,1960,Roma,ITA,402,88,36,18,34,4.038529e+10,Italy,5.019970e+07,Rome,Italy,1.0
124,1964,Tokyo,JPN,456,62,32,5,25,8.174901e+10,Japan,9.782600e+07,Tokyo,Japan,1.0
237,1968,Mexico City,MEX,446,9,3,3,3,2.936363e+10,Mexico,4.889402e+07,Mexico City,Mexico,1.0
309,1972,Munich,DEU,1041,253,74,83,96,2.986670e+11,Germany,7.868845e+07,Munich,Germany,1.0
414,1976,Montreal,CAN,531,23,0,9,14,2.065760e+11,Canada,2.351800e+07,Montreal,Canada,1.0
912,1992,Barcelona,ESP,537,69,48,19,2,6.292020e+11,Spain,3.915768e+07,Barcelona,Spain,1.0
1231,2000,Sydney,AUS,788,183,60,69,54,4.154460e+11,Australia,1.915300e+07,Sydney,Australia,1.0
1487,2004,Athina,GRC,499,31,8,18,5,2.405210e+11,Greece,1.095514e+07,Athens,Greece,1.0
1646,2008,Beijing,CHN,777,184,74,53,57,4.598210e+12,China,1.324655e+09,Beijing,China,1.0
2036,2016,Rio de Janeiro,BRA,583,50,36,8,6,1.796190e+12,Brazil,2.076529e+08,Rio de Janeiro,Brazil,1.0


In [77]:
country_data_complete = country_data_complete.drop(columns=["City_y", "Host_Country","No. of Athletes","Medal","Gold","Silver","Bronze"])
country_data_complete.head()

,Year,City_x,wb_country_code,GDP,Country,Population,Host
0,1960,Roma,AFG,5.377778e+08,Afghanistan,8996351.0,0.0
1,1960,Roma,ARG,NaN,Argentina,20619075.0,0.0
2,1960,Roma,AUS,1.859335e+10,Australia,10276477.0,0.0
3,1960,Roma,AUT,6.592694e+09,Austria,7047539.0,0.0
4,1960,Roma,BEL,1.165872e+10,Belgium,9153489.0,0.0


In [78]:
Country_Stats = country_data_complete.drop(columns=["Host","Country","City_x"])
Country_Stats.head()

,Year,wb_country_code,GDP,Population
0,1960,AFG,5.377778e+08,8996351.0
1,1960,ARG,NaN,20619075.0
2,1960,AUS,1.859335e+10,10276477.0
3,1960,AUT,6.592694e+09,7047539.0
4,1960,BEL,1.165872e+10,9153489.0


In [79]:
Country_Stats = Country_Stats.rename(columns={'wb_country_code':'Country_Code'})
Country_Stats.head()


,Year,Country_Code,GDP,Population
0,1960,AFG,5.377778e+08,8996351.0
1,1960,ARG,NaN,20619075.0
2,1960,AUS,1.859335e+10,10276477.0
3,1960,AUT,6.592694e+09,7047539.0
4,1960,BEL,1.165872e+10,9153489.0


In [80]:
Country_Stats = Country_Stats.rename(columns={'City_x':'City','wb_country_code':'Country_Code'})
Country_Stats.head()


,Year,Country_Code,GDP,Population
0,1960,AFG,5.377778e+08,8996351.0
1,1960,ARG,NaN,20619075.0
2,1960,AUS,1.859335e+10,10276477.0
3,1960,AUT,6.592694e+09,7047539.0
4,1960,BEL,1.165872e+10,9153489.0


In [81]:
Country = country_data_complete.drop(columns=["City_x","GDP","Population"])
Country.head()

,Year,wb_country_code,Country,Host
0,1960,AFG,Afghanistan,0.0
1,1960,ARG,Argentina,0.0
2,1960,AUS,Australia,0.0
3,1960,AUT,Austria,0.0
4,1960,BEL,Belgium,0.0


In [82]:
Country = Country.rename(columns={'wb_country_code':'Country_Code', 'Country':'Country_Name'})
Country.head()

,Year,Country_Code,Country_Name,Host
0,1960,AFG,Afghanistan,0.0
1,1960,ARG,Argentina,0.0
2,1960,AUS,Australia,0.0
3,1960,AUT,Austria,0.0
4,1960,BEL,Belgium,0.0


In [83]:
Country_Stats = Country_Stats.rename(columns={'wb_country_code':'Country_Code'})
Country_Stats.head()

,Year,Country_Code,GDP,Population
0,1960,AFG,5.377778e+08,8996351.0
1,1960,ARG,NaN,20619075.0
2,1960,AUS,1.859335e+10,10276477.0
3,1960,AUT,6.592694e+09,7047539.0
4,1960,BEL,1.165872e+10,9153489.0


In [84]:
Medals = is_host.drop(columns=["City_x", "No. of Athletes","GDP","Country","Population","City_y","Host_Country","Host"])
Medals.head()

,Year,wb_country_code,Medal,Gold,Silver,Bronze
37,1960,ITA,88,36,18,34
124,1964,JPN,62,32,5,25
237,1968,MEX,9,3,3,3
309,1972,DEU,253,74,83,96
414,1976,CAN,23,0,9,14


In [85]:
Medals = Medals.rename(columns={'Medal':'Total','wb_country_code':'Country_Code'})
Medals.head()

,Year,Country_Code,Total,Gold,Silver,Bronze
37,1960,ITA,88,36,18,34
124,1964,JPN,62,32,5,25
237,1968,MEX,9,3,3,3
309,1972,DEU,253,74,83,96
414,1976,CAN,23,0,9,14


In [86]:
##Connect to local Database##
rds_connection_string = "postgres:postgres@localhost:5432/db_project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [87]:
##Check for Tables##
engine.table_names()

['Country_Stats', 'Medals', 'Country', 'Country_Stats.to_sql', 'Medals.to_sql']

In [88]:
##Use pandas to load csv converted dataframes into database##
Country.to_sql(name='Country', con=engine, if_exists='append', index=False)

In [89]:
##Confirm data has been added by querying the tables

pd.read_sql_query('select * from "Country"', con=engine).head()

,Country_Code,Country_Name,Host,Year
0,AFG,Afghanistan,0,1960
1,ARG,Argentina,0,1960
2,AUS,Australia,0,1960
3,AUT,Austria,0,1960
4,BEL,Belgium,0,1960


In [90]:
Country_Stats.to_sql(name='Country_Stats', con=engine, if_exists='append', index=False)

In [91]:
pd.read_sql_query('select * from "Country_Stats"', con=engine).head()

,Country_Code,Population,GDP,Year,Host
0,AFG,8996351.0,537777811.0,1960,None
1,ARG,20619075.0,None,1960,None
2,AUS,10276477.0,18593347519.0,1960,None
3,AUT,7047539.0,6592693841.0,1960,None
4,BEL,9153489.0,11658722591.0,1960,None


In [92]:
Medals.to_sql(name='Medals', con=engine, if_exists='append', index=False)

In [93]:
pd.read_sql_query('select * from "Medals"', con=engine).head()

,Country_Code,Gold,Silver,Bronze,Total,Year
0,ITA,36,18,34,88,1960
1,JPN,32,5,25,62,1964
2,MEX,3,3,3,9,1968
3,DEU,74,83,96,253,1972
4,CAN,0,9,14,23,1976
